In [ ]:
pip install selenium


In [53]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from openpyxl.styles import Font, Alignment, Border, Side
import pandas as pd
import time
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment
import os
import numpy as np
import json


In [47]:
# Configurar o Selenium para rodar em segundo plano (headless)
chrome_options = Options()
chrome_options.add_argument("--headless")  # Rodar sem abrir o navegador
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Caminho para o ChromeDriver
driver_path = "C:/Users/Samsung/Documents/GitHub-Datas/Webscraping_Indicadores_Combustiveis/chromedriver-win64/chromedriver.exe"
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Acessar o site
driver.get("https://www.cepea.esalq.usp.br/br/indicador/etanol.aspx")

# Aguardar carregamento da página
time.sleep(3)

# Capturar o título da tabela
titulo = driver.find_element(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[1]").text

# Capturar os títulos das colunas
coluna_data = "DATA"
coluna_2 = driver.find_element(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/thead/tr/th[2]").text
coluna_3 = driver.find_element(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/thead/tr/th[3]").text
coluna_4 = driver.find_element(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/thead/tr/th[4]").text

# Criar a lista para armazenar os dados
dados = []

# Clicar no botão "Mais Dados" antes de coletar as informações
try:
    botao_mais_dados = driver.find_element(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[3]/a[3]")
    botao_mais_dados.click()
    time.sleep(3)  # Esperar o carregamento dos novos dados
except Exception as e:
    print("Botão 'Mais Dados' não encontrado ou erro ao clicar:", e)

# Identificar quantas linhas existem na tabela após carregar mais dados
num_linhas = len(driver.find_elements(By.XPATH, "/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/tbody/tr"))

# Iterar pelas linhas e capturar os valores das colunas
for i in range(1, num_linhas + 1):
    data = driver.find_element(By.XPATH, f"/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/tbody/tr[{i}]/td[1]").text
    valor_2 = driver.find_element(By.XPATH, f"/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/tbody/tr[{i}]/td[2]").text
    valor_3 = driver.find_element(By.XPATH, f"/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/tbody/tr[{i}]/td[3]").text
    valor_4 = driver.find_element(By.XPATH, f"/html/body/div/div[3]/div[2]/div[2]/div[2]/div[1]/div[2]/div[1]/table/tbody/tr[{i}]/td[4]").text
    
    # Adicionar ao dataset
    dados.append([data, valor_2, valor_3, valor_4])

# Criar o DataFrame com os nomes das colunas extraídas
df = pd.DataFrame(dados, columns=[coluna_data, coluna_2, coluna_3, coluna_4])

# Remover linhas vazias
df = df.replace("", None).dropna()

# Função para converter números no formato float
def converter_para_float(valor):
    if isinstance(valor, str):  # Verifica se é string
        valor = valor.strip()  # Remove espaços em branco extras
        if valor == "":  # Se for string vazia, retorna NaN
            return np.nan
        try:
            return float(valor.replace(".", "").replace(",", "."))  # Converte para float
        except ValueError:  # Se não puder converter, retorna NaN
            return np.nan
    return valor  # Se já for número, retorna o valor original

# Lista das colunas que devem ser mantidas como texto
colunas_texto = ["DATA","VAR./SEMANA"]

# Aplicar a conversão para todas as colunas, exceto as de texto
df = df.apply(lambda col: col.apply(converter_para_float) if col.name not in colunas_texto else col)

df = df.rename(columns={'VAR./SEMANA': 'VAR./SEMANA_%'})
df["VAR./SEMANA"] = df["VAR./SEMANA_%"].copy()

# Converter a coluna 'VAR./SEMANA' de string percentual para float decimal
df["VAR./SEMANA"] = df["VAR./SEMANA"].str.replace("%", "", regex=False)  # Remove o símbolo de %
df["VAR./SEMANA"] = df["VAR./SEMANA"].str.replace(",", ".", regex=False).astype(float) / 100  # Troca ',' por '.' e divide por 100

# Definir o caminho de salvamento
output_folder = "C:/Users/Samsung/Documents/GitHub-Datas/Webscraping_Indicadores_Combustiveis"
file_path = os.path.join(output_folder, "INDICADOR SEMANAL DO ETANOL HIDRATADO COMBUSTÍVEL.xlsx")

# Salvar em um arquivo Excel
df.to_excel(file_path, index=False, startrow=1)  # startrow=1 para deixar espaço para o título


# Ajustar o formato do Excel
wb = load_workbook(file_path)
ws = wb.active

# Mesclar as 5 primeiras colunas na linha 1 e adicionar o título
ws.merge_cells("A1:E1")
ws["A1"] = titulo

# Formatar a célula mesclada
ws["A1"].font = Font(bold=True, size=11)
ws["A1"].alignment = Alignment(horizontal="center", vertical="center",  wrap_text=True)

# Aumentar a altura da linha 1 (onde está a célula mesclada)
ws.row_dimensions[1].height = 30  # Ajuste o valor conforme necessário

# Criar borda para a célula mesclada
border_style = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin")
)

# Aplicar borda em todas as células mescladas (A1:E1)
for col in ["A", "B", "C", "D", "E"]:
    ws[f"{col}1"].border = border_style

# Ajustar largura das colunas (ignorar células mescladas)
for col in ws.iter_cols(min_row=2, max_row=ws.max_row):  # Começa da linha 2 para evitar erro na célula mesclada
    max_length = 0
    col_letter = col[0].column_letter  # Obter a letra da coluna
    for cell in col:
        if cell.value and isinstance(cell.value, str):
            max_length = max(max_length, len(cell.value))
    ws.column_dimensions[col_letter].width = max_length + 2

# Salvar as alterações
wb.save(file_path)

# Fechar o Selenium
driver.quit()

print(f"Arquivo salvo com sucesso em: {file_path}")


In [54]:

# Converter o DataFrame para JSON no formato 'records'
df_json = df.to_json(orient='records', date_format='iso')

# Definir o caminho de salvamento para o arquivo JSON
json_output_path = "C:/Users/Samsung/Documents/GitHub-Datas/Webscraping_Indicadores_Combustiveis/indicador_etanol.json"

# Salvar o JSON em um arquivo
with open(json_output_path, 'w') as json_file:
    json.dump(json.loads(df_json), json_file, indent=4)

print(f"Arquivo JSON salvo com sucesso em: {json_output_path}")

Arquivo JSON salvo com sucesso em: C:/Users/Samsung/Documents/GitHub-Datas/Webscraping_Indicadores_Combustiveis/indicador_etanol.json
